In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import faststt, fastspeech, transducer
import tensorflow as tf






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
config = malaya_speech.config.fastspeech2_config_v2
config = fastspeech.Config(vocab_size = 1, **config)

In [5]:
model = faststt.Model(config.encoder_self_attention_params)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
config = malaya_speech.config.conformer_base_decoder_config
small_transducer = transducer.rnn.Model(model, vocabulary_size = 100, **config)

In [7]:
config = malaya_speech.config.transducer_featurizer_config
featurizer = malaya_speech.utils.tf_featurization.STTFeaturizer(**config)

In [8]:
k = 0.0062169171196134855

i = tf.placeholder(tf.float32, [None, None])
lengths = tf.placeholder(tf.float32, [None])
v = tf.expand_dims(tf.map_fn(featurizer.vectorize, i), -1)
length_v = tf.cast(lengths * k, tf.int32)

In [9]:
out = model(v)
out.shape

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


TensorShape([Dimension(None), Dimension(None), Dimension(256)])

In [10]:
p = tf.placeholder(tf.int32, [None, None])
p_len = tf.placeholder(tf.int32, [None])

small_logits = small_transducer([v, p, p_len], training = True)
small_logits


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


<tf.Tensor 'transducer/transducer_joint/transducer_joint_vocab/BiasAdd:0' shape=(?, ?, ?, 100) dtype=float32>

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [12]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

(90090, 56298)

In [13]:
padded, lens = malaya_speech.padding.sequence_1d([y, y1], return_len = True)
t = [[1,1,1,1,1,1], [1,1,1,1,1,1]]

In [16]:
# %%time

# sess.run(out, feed_dict = {i: y}).shape

In [17]:
# %%time

# sess.run(out, feed_dict = {i: y1}).shape

In [21]:
%%time

sess.run(small_logits, feed_dict = {i: padded, lengths: lens, p: t, p_len: [6, 6]}).shape

CPU times: user 302 ms, sys: 114 ms, total: 416 ms
Wall time: 97.9 ms


(2, 141, 6, 100)

In [ ]:
saver = tf.train.Saver()

In [ ]:
saver.save(sess, 'test/model.ckpt')

In [ ]:
!ls -lh test